In [1]:
from glob import glob
import fiona
import geopandas as gpd
import pandas as pd
from cartoframes.auth import set_default_credentials
from cartoframes import to_carto, has_table, delete_table
import georasters as gr
import numpy as np
from shapely.geometry import MultiPolygon
from shapely import wkt, wkb
import shapely.ops

failed to import boto3, continuing.


/Users/carstenrodin/anaconda3/envs/gis/lib/python3.7/site-packages/pysal/explore/segregation/network/network.py:16: UserWarning: You need pandana and urbanaccess to work with segregation's network module
You can install them with  `pip install urbanaccess pandana` or `conda install -c udst pandana urbanaccess`
  "You need pandana and urbanaccess to work with segregation's network module\n"
/Users/carstenrodin/anaconda3/envs/gis/lib/python3.7/site-packages/pysal/model/spvcm/abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


In [2]:
# set authentication from local file for cartoframes
set_default_credentials("/Users/carstenrodin/keys/2020-02-05_dcp-carto-upload.json")

In [6]:
# can change the table here to test connection
# has_table('doitt_bldgs_test')

False

## Import Buildings
from https://github.com/CityOfNewYork/nyc-geo-metadata/blob/master/Metadata/Metadata_3DBuildingModel.md

In [4]:
# load buildings from each separate file-geodatabase in the search path into a list
bldg_gdf_list = []
bldg_gdb_paths = glob("./in/DA_WISE_Multipatch/DA_WISE_Multipatch/*/")
for i in bldg_gdb_paths:
    print(i)
    i_gdf = gpd.read_file(i, driver='FileGDB', layer='Buildings_3D_Multipatch')
    bldg_gdf_list.append(i_gdf.to_crs(epsg=4326)) # set crs as EPSG:4326

./in/DA_WISE_Multipatch/DA_WISE_Multipatch/DA14_3D_Buildings_Multipatch.gdb/
./in/DA_WISE_Multipatch/DA_WISE_Multipatch/DA11_3D_Buildings_Multipatch.gdb/
./in/DA_WISE_Multipatch/DA_WISE_Multipatch/DA1_3D_Buildings_Multipatch.gdb/
./in/DA_WISE_Multipatch/DA_WISE_Multipatch/DA4_3D_Buildings_Multipatch.gdb/
./in/DA_WISE_Multipatch/DA_WISE_Multipatch/DA10_3D_Buildings_Multipatch.gdb/
./in/DA_WISE_Multipatch/DA_WISE_Multipatch/DA15_3D_Buildings_Multipatch.gdb/
./in/DA_WISE_Multipatch/DA_WISE_Multipatch/DA5_3D_Buildings_Multipatch.gdb/
./in/DA_WISE_Multipatch/DA_WISE_Multipatch/DA20_3D_Buildings_Multipatch.gdb/
./in/DA_WISE_Multipatch/DA_WISE_Multipatch/DA3_3D_Buildings_Multipatch.gdb/
./in/DA_WISE_Multipatch/DA_WISE_Multipatch/DA6_3D_Buildings_Multipatch.gdb/
./in/DA_WISE_Multipatch/DA_WISE_Multipatch/DA19_3D_Buildings_Multipatch.gdb/
./in/DA_WISE_Multipatch/DA_WISE_Multipatch/DA16_3D_Buildings_Multipatch.gdb/
./in/DA_WISE_Multipatch/DA_WISE_Multipatch/DA9_3D_Buildings_Multipatch.gdb/
./in/

In [5]:
bldgs_gdf = pd.concat(bldg_gdf_list) 
bldgs_gdf['BIN'].size # check size

1083529

In [6]:
# make 2d convex hulls of each geometry to use as the primary the_geom field in Carto
# this will be used for spatial queries
bldgs_gdf['the_geom'] = bldgs_gdf['geometry'].convex_hull.apply(lambda g: shapely.ops.transform(lambda x, y, z=None: (x, y), g ) )

In [7]:
# then make a separate field with the 3d geometry as well-known text 
# these will be of multipolygon Z type in EPSG:2263 crs
bldgs_gdf['g3d'] = bldgs_gdf['geometry'].to_crs(epsg=2263).apply(lambda g: str(g.wkt) )

In [9]:
# split into chunked list
size = 10000
upload_list = [bldgs_gdf.iloc[i:i+size-1,:] for i in range(0, len(bldgs_gdf),size)]

In [18]:
# upload chunks with specified fields only: BIN, geometry and 3d geometry
for uf in upload_list:
    to_carto(uf[['the_geom', 'BIN', 'g3d']], 'doitt_3dbldgs_v2016', if_exists="append")

Success! Data uploaded to table "doitt_3dbldgs_v2016" correctly
Success! Data uploaded to table "doitt_3dbldgs_v2016" correctly
Success! Data uploaded to table "doitt_3dbldgs_v2016" correctly
Success! Data uploaded to table "doitt_3dbldgs_v2016" correctly
Success! Data uploaded to table "doitt_3dbldgs_v2016" correctly
Success! Data uploaded to table "doitt_3dbldgs_v2016" correctly
Success! Data uploaded to table "doitt_3dbldgs_v2016" correctly
Success! Data uploaded to table "doitt_3dbldgs_v2016" correctly
Success! Data uploaded to table "doitt_3dbldgs_v2016" correctly
Success! Data uploaded to table "doitt_3dbldgs_v2016" correctly
Success! Data uploaded to table "doitt_3dbldgs_v2016" correctly
Success! Data uploaded to table "doitt_3dbldgs_v2016" correctly
Success! Data uploaded to table "doitt_3dbldgs_v2016" correctly
Success! Data uploaded to table "doitt_3dbldgs_v2016" correctly
Success! Data uploaded to table "doitt_3dbldgs_v2016" correctly
Success! Data uploaded to table "doitt_3

## Import Street Tree Points
see: https://data.cityofnewyork.us/Environment/2015-Street-Tree-Census-Tree-Data/pi5s-9p35  

with geometry, coded species and diameter

In [7]:
street_tree_path = "/Users/carstenrodin/local-git/ud-3d-model/in/2015 Street Tree Census - Tree Data.geojson"
street_tree_gdf = gpd.read_file(street_tree_path)

In [8]:
# fill na values for species
street_tree_gdf['spc_common'].fillna('unknown', inplace=True)

In [9]:
# code tree species as integer
treecode_dict = {tree: i+1 for i, tree in enumerate(sorted(list(street_tree_gdf['spc_common'].unique())))}

In [10]:
# create coded field for species instead of string
street_tree_gdf['spc_code'] = street_tree_gdf.apply(lambda row: treecode_dict[row['spc_common']], axis=1)

In [ ]:
# show available columns
# street_tree_gdf.columns

In [ ]:
# upload to carto, select fields only
to_carto(street_tree_gdf[['geometry', 'spc_code', 'tree_dbh']], 'dpr_streettrees_v2016', if_exists="replace")

## Import Elevation

This is currently done by sampling a DEM raster from a regularly spaced grid of points in QGIS, then uploading those points

In [ ]:
street_intersection_path = "/Users/carstenrodin/local-git/ud-3d-model/in/mn_regularpts_100/mn_regularpts_100_raster.shp"
street_intersection_gdf = gpd.read_file(street_intersection_path).to_crs(epsg=4326)

In [ ]:
street_intersection_gdf

In [ ]:
to_carto(street_intersection_gdf, 'ud_elevationpts_v2020', if_exists="replace")

## Import Planimetrics
see https://github.com/CityOfNewYork/nyc-planimetrics/blob/master/Capture_Rules.md

- [x] hydrography as geom only
- [x] roadbed as geom only
- [x] sidewalk as geom only

In [ ]:
# load planimetrics as EPSG:4326
planimetric_gdf_list = {}
planimetric_path = './in/NYC_DoITT_Planimetric_OpenData.gdb/'
planimetric_layers = fiona.listlayers(planimetric_path)
for l in planimetric_layers:
    l_gdf = gpd.read_file(planimetric_path, driver='FileGDB', layer=l)
    planimetric_gdf_list[l] = l_gdf.to_crs(epsg=4326)

In [ ]:
# planimetric_gdf_list.keys()

In [ ]:
# planimetric_gdf_list['SIDEWALK'].head()

In [ ]:
# upload to carto
to_carto(planimetric_gdf_list['SIDEWALK'][['geometry']], 'doitt_sidewalk', if_exists="replace")

## Other Export Options

In [ ]:
# save to file
# bldgs_gdf.to_file("/Users/carstenrodin/Desktop/bldgs.gpkg", driver="GPKG")